# Data Cleaning - TOC-based Heading Hierarchy (v1)

**Notebook ID:** `02_data_cleaning_v1`
**Description:** Use Table of Contents as source of truth for heading hierarchy and casing

This version:
- Extracts TOC entries to establish authoritative heading structure
- Uses fuzzy matching to handle variations (case-insensitive with ~0.85 threshold)
- Applies exact TOC casing to matched headings
- Converts unmatched H2 headings to bold emphasis
- Provides detailed matching statistics

---


In [48]:
# # CELL_ID: config_v1
# from pathlib import Path

# # Configuration
# PDF_NAME = "Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1"
# OUTPUT_DIR = Path("output")
# OUTPUT_DIR.mkdir(exist_ok=True)


## 1. Extract Table of Contents

Parse the TABLE OF CONTENT section to extract numbered entries and their titles.
This creates our authoritative mapping for heading structure and casing.


In [49]:
# CELL_ID: extract_toc
import re
from pathlib import Path
from collections import defaultdict

def extract_toc_entries(markdown_path):
    """
    Extract numbered section entries from the Table of Contents.
    
    Returns:
        Dictionary mapping section numbers to titles: {
            '1.1.': 'Definition',
            '1.2.1.': 'Pathogenesis and pathophysiology of type 1 diabetes',
            etc.
        }
    """
    markdown_path = Path(markdown_path)
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # Find the TABLE OF CONTENT section
    toc_start = None
    toc_end = None
    
    for i, line in enumerate(lines):
        if line.strip() == "# TABLE OF CONTENT" or line.strip() == "## TABLE OF CONTENT":
            toc_start = i
        elif toc_start is not None and line.strip().startswith("# ") and "TABLE OF CONTENT" not in line:
            toc_end = i
            break
    
    if toc_start is None:
        print("⚠ WARNING: TABLE OF CONTENT section not found")
        return {}
    
    if toc_end is None:
        toc_end = len(lines)
    
    # Pattern to match table rows: | number | title | page |
    # Captures numbered sections like "1.1.", "2.3.4", "3.2.1", etc.
    numbered_pattern = re.compile(r'^\s*\|\s*((?:\d+\.)+\d+\.?)\s*\|\s*(.+?)\s*\|\s*\d+\s*\|$')
    
    toc_entries = {}
    
    for i in range(toc_start, toc_end):
        line = lines[i].strip()
        
        # Skip table separators and headers
        if line.startswith("---") or not line or line.startswith("#"):
            continue
        
        # Try to match numbered pattern
        match = numbered_pattern.match(line)
        if match:
            number = match.group(1).strip()
            title = match.group(2).strip()
            
            # Normalize: ensure trailing period for consistency
            if not number.endswith("."):
                number += "."
            
            toc_entries[number] = title
    
    return toc_entries


# Test TOC extraction
test_file = "output/Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited.md"
toc_entries = extract_toc_entries(test_file)

print("="*60)
print("TOC EXTRACTION TEST")
print("="*60)
print(f"\n✓ Extracted {len(toc_entries)} numbered entries from TOC")
print(f"\n📋 Sample entries:")
for i, (number, title) in enumerate(list(toc_entries.items())[:10]):
    print(f"  {number}: {title}")
if len(toc_entries) > 10:
    print(f"  ... and {len(toc_entries) - 10} more")


TOC EXTRACTION TEST

✓ Extracted 68 numbered entries from TOC

📋 Sample entries:
  1.1.: Definition
  1.2.: Pathophysiology
  1.2.1.: Pathogenesis and pathophysiology of type 1 diabetes
  1.2.2.: Pathogenesis and pathophysiology of type 2 diabetes
  1.3.: Diagnosis of diabetes
  1.4.: Classification of Diabetes Mellitus
  1.5.: Risk Factors
  1.6.: Screening for Diabetes Mellitus type 2
  2.0.: Introduction
  2.1.: Management of Type 1 Diabetes
  ... and 58 more


## 2. Fuzzy Matching

Implement fuzzy matching to handle variations in heading text while matching case-insensitively.


In [50]:
# CELL_ID: fuzzy_match
from difflib import SequenceMatcher
import re

def fuzzy_match(text1, text2, threshold=0.85):
    """
    Perform fuzzy matching between two texts using SequenceMatcher.
    
    Args:
        text1: First text to compare
        text2: Second text to compare
        threshold: Minimum similarity score to consider a match (0.0-1.0)
    
    Returns:
        Similarity score (0.0-1.0) or None if below threshold
    """
    if not text1 or not text2:
        return None
    
    # Normalize: lowercase for comparison, remove extra whitespace
    norm1 = re.sub(r'\s+', ' ', text1.lower().strip())
    norm2 = re.sub(r'\s+', ' ', text2.lower().strip())
    
    # Calculate similarity using SequenceMatcher
    similarity = SequenceMatcher(None, norm1, norm2).ratio()
    
    return similarity if similarity >= threshold else None


def find_best_toc_match(heading_text, toc_entries, threshold=0.85):
    """
    Find the best matching TOC entry for a given heading text.
    
    Returns:
        Tuple of (section_number, title, similarity_score) or (None, None, None) if no match
    """
    best_match = None
    best_score = 0
    
    # Normalize heading text for comparison
    norm_heading = re.sub(r'\s+', ' ', heading_text.strip())
    
    for number, title in toc_entries.items():
        score = fuzzy_match(norm_heading, title, threshold)
        if score is not None and score > best_score:
            best_score = score
            best_match = (number, title, score)
    
    if best_match and best_score >= threshold:
        return best_match
    
    return (None, None, None)


# Test fuzzy matching
test_cases = [
    ("Definition", "definition", 0.85),
    ("Management of Type 1 Diabetes", "Management of Type 1 Diabetes", 0.85),
    ("Types of Insulin", "Type of Insulin", 0.85),
    ("Blood glucose monitoring", "Blood Glucose Monitoring", 0.85),
    ("Completely different", "Definition", 0.85),
]

print("="*60)
print("FUZZY MATCHING TEST")
print("="*60)
for text1, text2, threshold in test_cases:
    result = fuzzy_match(text1, text2, threshold)
    status = f"✓ Match: {result:.2f}" if result else "✗ No match"
    print(f"  '{text1}' vs '{text2}': {status}")


FUZZY MATCHING TEST
  'Definition' vs 'definition': ✓ Match: 1.00
  'Management of Type 1 Diabetes' vs 'Management of Type 1 Diabetes': ✓ Match: 1.00
  'Types of Insulin' vs 'Type of Insulin': ✓ Match: 0.97
  'Blood glucose monitoring' vs 'Blood Glucose Monitoring': ✓ Match: 1.00
  'Completely different' vs 'Definition': ✗ No match


## 3. Main Function: Fix Hierarchy with TOC

Match document headings against TOC entries, apply TOC casing, and convert unmatched H2s to emphasis.


In [51]:
# CELL_ID: main_fix

def fix_hierarchy_with_toc(markdown_path, toc_entries, output_path=None, fuzzy_threshold=0.85):
    """
    Fix heading hierarchy using Table of Contents as source of truth.
    
    Process:
    1. Extract headings from document
    2. Handle special sections (TOC, PREFACE, etc.) → H1
    3. Handle chapters (CHAPTER ONE + next heading) → H1
    4. Match numbered headings against TOC using number pattern + fuzzy text matching
    5. Apply TOC casing to matched headings
    6. Convert unmatched non-numbered headings to bold emphasis
    7. Track and report statistics
    
    Returns:
        Tuple of (output_path, statistics_dict)
    """
    markdown_path = Path(markdown_path)
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # Pattern to match numbered headings: "1.1.", "2.3.4.", etc.
    numbered_pattern = re.compile(r'^(#{1,6})\s+((?:\d+\.)+\d+\.?)\s+(.+?)\s*$')
    
    # Pattern to match chapters: "CHAPTER ONE", "CHAPTER TWO", etc.
    chapter_pattern = re.compile(r'^(#{1,6})\s+CHAPTER\s+(ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE)', re.IGNORECASE)
    
    # Major document sections that should be H1 (top-level document structure)
    major_sections = ['TABLE OF CONTENT', 'LIST OF FIGURES', 'LIST OF TABLES', 
                     'ACRONYMS', 'FOREWORD', 'PREFACE', 'ACKNOWLEDGEMENTS', 
                     'EXECUTIVE SUMMARY', 'REFERENCES', 'APPENDICES']
    
    # Storage for processing
    fixed_lines = []
    statistics = {
        'total_headings': 0,
        'matched_by_number': 0,
        'matched_by_text': 0,
        'matched_total': 0,
        'h2_to_emphasis': 0,
        'chapters_combined': 0,
        'special_sections_h1': 0,
        'kept_h3_h4_h5': 0,
        'toc_not_found': set(),
    }
    
    # Process each line
    i = 0
    while i < len(lines):
        line = lines[i]
        stripped = line.strip()
        
        # Check if this is a heading
        if stripped.startswith('#'):
            heading_text = stripped.lstrip('#').strip()
            heading_level = stripped[:len(stripped) - len(stripped.lstrip('#'))]
            
            # ====================================================================
            # RULE 1: CHAPTER HEADINGS → H1 (combined with next heading)
            # ====================================================================
            chapter_match = chapter_pattern.match(stripped)
            if chapter_match:
                # Look ahead for the next heading (usually immediately after CHAPTER)
                # This is typically the chapter title/introduction
                found_next = False
                for j in range(i + 1, min(i + 5, len(lines))):  # Check next 4 lines
                    next_line = lines[j].strip()
                    
                    if next_line.startswith('#'):
                        next_heading_text = next_line.lstrip('#').strip()
                        
                        # Don't combine with another CHAPTER or numbered section
                        # We want to combine with the chapter title/intro
                        if not chapter_pattern.match(next_line) and not numbered_pattern.match(next_line):
                            # Combine into single H1: "CHAPTER ONE: INTRODUCTION TO DIABETES"
                            combined = f"# {stripped.lstrip('#').strip()}: {next_heading_text}\n"
                            fixed_lines.append(combined)
                            statistics['chapters_combined'] += 1
                            statistics['total_headings'] += 1
                            
                            # Skip the next heading line (we've merged it)
                            i = j  # Jump past the next heading
                            found_next = True
                            break
                
                # If no suitable next heading found, just convert CHAPTER to H1
                if not found_next:
                    fixed_lines.append(stripped.replace(heading_level, "#", 1) + "\n")
                    statistics['chapters_combined'] += 1
                    statistics['total_headings'] += 1
                
                i += 1
                continue  # Move to next line
            
            # ====================================================================
            # RULE 2: MAJOR DOCUMENT SECTIONS → H1
            # ====================================================================
            elif any(section in heading_text.upper() for section in major_sections):
                fixed_lines.append(line.replace(heading_level, "#", 1))  # Convert to H1
                statistics['special_sections_h1'] += 1
                statistics['total_headings'] += 1
                i += 1
                continue
            
            # ====================================================================
            # RULE 3: NUMBERED PATTERNS → Match with TOC
            # ====================================================================
            else:
                # Try to match numbered heading pattern
                match = numbered_pattern.match(stripped)
                if match:
                    statistics['total_headings'] += 1
                    
                    number = match.group(2).strip()
                    heading_text = match.group(3).strip()
                    
                    # Normalize: ensure trailing period
                    if not number.endswith("."):
                        number += "."
                    
                    # Calculate heading level based on number pattern
                    # 2 numbers (1.1.) → H2, 3 numbers (1.1.1.) → H3, 4+ → H4
                    num_count = len([x for x in number.split('.') if x])
                    if num_count == 2:
                        calculated_level = '##'  # H2
                    elif num_count == 3:
                        calculated_level = '###'  # H3
                    elif num_count >= 4:
                        calculated_level = '####'  # H4
                    else:
                        calculated_level = heading_level  # Fallback for unusual patterns
                    
                    # Check if this number exists in TOC
                    if number in toc_entries:
                        statistics['matched_by_number'] += 1
                        toc_title = toc_entries[number]
                        
                        # Determine if text needs fuzzy matching
                        norm_doc = re.sub(r'\s+', ' ', heading_text.lower().strip())
                        norm_toc = re.sub(r'\s+', ' ', toc_title.lower().strip())
                        
                        if norm_doc != norm_toc:
                            # Text differs, but number matches - apply TOC casing
                            # Use fuzzy match to verify they're actually the same
                            if fuzzy_match(norm_doc, norm_toc, fuzzy_threshold):
                                statistics['matched_by_text'] += 1
                                # Apply exact TOC casing
                                heading_text = toc_title
                        
                        statistics['matched_total'] += 1
                        
                        # Reconstruct heading with correct level and casing
                        fixed_lines.append(f"{calculated_level} {number} {heading_text}\n")
                    
                    else:
                        # Number not in TOC - check if it's a subsection to keep
                        num_count = len([x for x in number.split('.') if x])
                        
                        # Calculate heading level for unmatched entries too
                        if num_count == 2:
                            calculated_level = '##'  # H2
                        elif num_count == 3:
                            calculated_level = '###'  # H3
                        elif num_count >= 4:
                            calculated_level = '####'  # H4
                        else:
                            calculated_level = heading_level
                        
                        if num_count == 2:
                            # This is an H2 (like 2.1.) but not in TOC
                            # Convert to bold emphasis
                            statistics['h2_to_emphasis'] += 1
                            fixed_lines.append(f"**{number} {heading_text}**\n")
                        else:
                            # H3, H4, H5 - keep as heading (subsections) with calculated level
                            statistics['kept_h3_h4_h5'] += 1
                            fixed_lines.append(f"{calculated_level} {number} {heading_text}\n")
                        
                        statistics['toc_not_found'].add(number)
                
                else:
                    # Heading without number pattern (like ## Introduction)
                    # This is NOT a chapter, NOT a special section, and NOT numbered
                    # Convert to bold emphasis
                    statistics['h2_to_emphasis'] += 1
                    statistics['total_headings'] += 1
                    fixed_lines.append(f"**{heading_text}**\n")
        
        else:
            # Not a heading - keep as-is
            fixed_lines.append(line)
        
        i += 1
    
    # Save fixed markdown
    if output_path is None:
        output_path = markdown_path.parent / f"{markdown_path.stem}_hierarchy_fixed.md"
    else:
        output_path = Path(output_path)
    
    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(fixed_lines)
    
    # Convert set to list for JSON serialization if needed
    statistics['toc_not_found'] = sorted(list(statistics['toc_not_found']))
    
    return str(output_path), statistics


## 4. Execute TOC-based Fixing


In [52]:
# CELL_ID: execute

# Load input file
input_file = "output/Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited.md"

# Extract TOC entries
print("="*60)
print("STEP 1: EXTRACTING TABLE OF CONTENTS")
print("="*60)
toc_entries = extract_toc_entries(input_file)
print(f"\n✓ Found {len(toc_entries)} numbered entries in TOC")

# Apply TOC-based fixing
print(f"\n{'='*60}")
print("STEP 2: APPLYING TOC-BASED HIERARCHY FIXES")
print(f"{'='*60}")
output_file, stats = fix_hierarchy_with_toc(input_file, toc_entries, fuzzy_threshold=0.85)

print(f"\n✓ Processing complete")
print(f"✓ Saved to: {output_file}")

# Display statistics
print(f"{'='*60}")
print("MATCHING STATISTICS")
print(f"{'='*60}")
print(f"📊 Summary:")
print(f"  • Total headings processed: {stats['total_headings']}")
print(f"  • Chapters combined (H1): {stats['chapters_combined']}")
print(f"  • Special sections (H1): {stats['special_sections_h1']}")
print(f"  • Matched by number: {stats['matched_by_number']}")
print(f"  • Additional text corrections: {stats['matched_by_text']}")
print(f"  • Total matched: {stats['matched_total']}")
print(f"  • Converted to emphasis: {stats['h2_to_emphasis']}")
print(f"  • Kept as H3/H4/H5: {stats['kept_h3_h4_h5']}")

if stats['toc_not_found']:
    print(f"\n⚠ TOC entries not found in document: {len(stats['toc_not_found'])}")
    for number in stats['toc_not_found'][:10]:
        print(f"    {number}")
    if len(stats['toc_not_found']) > 10:
        print(f"    ... and {len(stats['toc_not_found']) - 10} more")


STEP 1: EXTRACTING TABLE OF CONTENTS

✓ Found 68 numbered entries in TOC

STEP 2: APPLYING TOC-BASED HIERARCHY FIXES

✓ Processing complete
✓ Saved to: output\Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited_hierarchy_fixed.md
MATCHING STATISTICS
📊 Summary:
  • Total headings processed: 315
  • Chapters combined (H1): 8
  • Special sections (H1): 10
  • Matched by number: 69
  • Additional text corrections: 6
  • Total matched: 69
  • Converted to emphasis: 228
  • Kept as H3/H4/H5: 0


## 3.5. Markdown Quality Cleaning

Apply quality checks and cleaning based on analysis:
- Remove OCR errors (corrupted lines)
- Remove trailing whitespace
- Limit consecutive empty lines (max 2)
- Normalize excessive spaces (preserve tables)
- Fix heading spacing


In [53]:
# CELL_ID: 02_data_cleaning_a2db9ebe
from pathlib import Path
import re
from collections import Counter, defaultdict

def analyze_markdown_quality(markdown_path):
    """
    Analyze markdown file to identify formatting issues and inconsistencies.
    
    Returns:
        Dictionary containing total_lines, total_chars, issues dict, and statistics dict
    """
    markdown_path = Path(markdown_path)
    
    if not markdown_path.exists():
        raise FileNotFoundError(f"Markdown file not found: {markdown_path}")
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    analysis = {
        "total_lines": len(lines),
        "total_chars": sum(len(l) for l in lines),
        "issues": defaultdict(list),
        "statistics": defaultdict(int),
    }
    
    consecutive_empty = 0
    
    for i, line in enumerate(lines, 1):
        stripped = line.strip()
        
        # Check for trailing whitespace
        if line != line.rstrip('\n') and line.rstrip('\n') != line.rstrip():
            analysis["issues"]["trailing_whitespace"].append(i)
        
        # Track empty lines
        if not stripped:
            analysis["statistics"]["empty_lines"] += 1
            consecutive_empty += 1
            if consecutive_empty > 2:
                analysis["issues"]["excessive_empty_lines"].append({
                    "line": i,
                    "count": consecutive_empty
                })
        else:
            consecutive_empty = 0
            
            # Detect single character lines (OCR errors)
            if len(stripped) == 1 and stripped.isalpha():
                if i > 2 and len(lines[i-2].strip()) <= 2:
                    analysis["issues"]["ocr_errors"].append({
                        "line": i,
                        "text": stripped
                    })
            
            # Count headings and check formatting
            if line.startswith("#"):
                level = len(line) - len(line.lstrip("#"))
                analysis["statistics"][f"H{level}"] += 1
                if re.search(r'\s{3,}', stripped):
                    analysis["issues"]["heading_formatting"].append(i)
            
            # Validate table formatting
            if "|" in line and line.strip().startswith("|"):
                pipe_count = line.count("|")
                if pipe_count < 3:
                    analysis["issues"]["malformed_tables"].append({
                        "line": i,
                        "pipe_count": pipe_count
                    })
            
            # Detect excessive spaces (outside tables)
            if re.search(r' {4,}', stripped) and "|" not in line:
                analysis["issues"]["excessive_spaces"].append(i)
            
            # Validate image links
            img_pattern = r'!\[([^\]]*)\]\(([^)]+)\)'
            for match in re.finditer(img_pattern, line):
                alt_text, path = match.groups()
                if not alt_text.strip():
                    analysis["issues"]["empty_image_alt"].append(i)
                if not path.startswith("images/") and not path.startswith("http"):
                    analysis["issues"]["broken_image_links"].append({
                        "line": i,
                        "path": path[:50]
                    })
    
    return analysis


def clean_markdown_based_on_analysis(markdown_path, analysis, output_path=None):
    """
    Clean markdown file based on issues identified in analysis.
    
    Returns:
        Tuple of (output_file_path, changes_dict)
    """
    markdown_path = Path(markdown_path)
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    cleaned_lines = []
    changes = defaultdict(int)
    skip_lines = set()
    
    # Mark corrupted lines for removal
    for issue in analysis["issues"].get("ocr_errors", []):
        skip_lines.add(issue["line"] - 1)
        changes["removed_corrupted"] += 1
    
    consecutive_empty = 0
    
    for i, line in enumerate(lines):
        if i in skip_lines:
            continue
        
        original = line
        
        # Remove trailing whitespace
        if line != line.rstrip('\n'):
            line = line.rstrip('\n').rstrip() + '\n'
            if line != original:
                changes["trailing_whitespace"] += 1
        
        # Handle empty lines (limit to max 2 consecutive)
        if not line.strip():
            consecutive_empty += 1
            if consecutive_empty <= 2:
                cleaned_lines.append(line)
            else:
                changes["excessive_empty_lines"] += 1
                continue
        else:
            consecutive_empty = 0
            
            # Normalize excessive spaces (preserve tables)
            if "|" not in line and re.search(r' {4,}', line):
                line = re.sub(r' {4,}', '   ', line)
                changes["excessive_spaces"] += 1
            
            # Fix heading spacing
            if line.strip().startswith("#"):
                heading = line.lstrip("#").strip()
                heading_fixed = re.sub(r'\s+', ' ', heading)
                if heading != heading_fixed:
                    prefix = "#" * (len(line) - len(line.lstrip("#")))
                    line = prefix + " " + heading_fixed + "\n"
                    changes["heading_spacing"] += 1
        
        cleaned_lines.append(line)
    
    # Save cleaned markdown
    if output_path is None:
        output_path = markdown_path.parent / f"{markdown_path.stem}_final.md"
    else:
        output_path = Path(output_path)
    
    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(cleaned_lines)
    
    return str(output_path), changes




In [54]:
# CELL_ID: execute_cleaning

# Apply markdown quality cleaning
print("="*60)
print("STEP 3: APPLYING MARKDOWN QUALITY CLEANING")
print("="*60)

# Analyze the output from TOC-based fixing
analysis = analyze_markdown_quality(output_file)

print(f"\n📊 Quality Analysis:")
print(f"  • Total lines: {analysis['total_lines']:,}")
print(f"  • Total characters: {analysis['total_chars']:,}")
print(f"  • Empty lines: {analysis['statistics']['empty_lines']:,}")

print(f"\n📑 Heading Counts:")
for key in sorted([k for k in analysis['statistics'].keys() if k.startswith('H')]):
    print(f"  • {key}: {analysis['statistics'][key]}")

# Display issues found
print(f"\n🔍 Issues Found:")
total_issues = 0
for issue_type, issues in analysis['issues'].items():
    count = len(issues)
    if count > 0:
        print(f"  ⚠ {issue_type}: {count}")
        total_issues += count

if total_issues == 0:
    print("  ✓ No issues detected!")

# Clean the markdown
from pathlib import Path
final_output = Path(output_file).parent / f"{Path(output_file).stem.split('_hierarchy_fixed')[0]}_final.md"
cleaned_file, cleaning_changes = clean_markdown_based_on_analysis(output_file, analysis, output_path=str(final_output))

print(f"\n{'='*60}")
print("CLEANING COMPLETE")
print(f"{'='*60}")
print(f"\n✓ Cleaning changes:")
for change_type, count in sorted(cleaning_changes.items()):
    if count > 0:
        print(f"  • {change_type}: {count}")

print(f"\n✓ Final cleaned file saved to: {cleaned_file}")
print(f"\n📝 Output file: {cleaned_file}")


STEP 3: APPLYING MARKDOWN QUALITY CLEANING

📊 Quality Analysis:
  • Total lines: 4,150
  • Total characters: 356,708
  • Empty lines: 1,331

📑 Heading Counts:
  • H1: 18
  • H2: 38
  • H3: 31

🔍 Issues Found:
  ⚠ ocr_errors: 78
  ⚠ excessive_spaces: 3

CLEANING COMPLETE

✓ Cleaning changes:
  • excessive_empty_lines: 59
  • excessive_spaces: 3
  • removed_corrupted: 78

✓ Final cleaned file saved to: output\Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited_final.md

📝 Output file: output\Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited_final.md


## 3.5. Markdown Quality Cleaning

Apply quality checks and cleaning based on analysis:
- Remove OCR errors (corrupted lines)
- Remove trailing whitespace
- Limit consecutive empty lines (max 2)
- Normalize excessive spaces (preserve tables)
- Fix heading spacing


In [55]:
# CELL_ID: 02_data_cleaning_a2db9ebe
from pathlib import Path
import re
from collections import Counter, defaultdict

def analyze_markdown_quality(markdown_path):
    """
    Analyze markdown file to identify formatting issues and inconsistencies.
    
    Returns:
        Dictionary containing total_lines, total_chars, issues dict, and statistics dict
    """
    markdown_path = Path(markdown_path)
    
    if not markdown_path.exists():
        raise FileNotFoundError(f"Markdown file not found: {markdown_path}")
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    analysis = {
        "total_lines": len(lines),
        "total_chars": sum(len(l) for l in lines),
        "issues": defaultdict(list),
        "statistics": defaultdict(int),
    }
    
    consecutive_empty = 0
    
    for i, line in enumerate(lines, 1):
        stripped = line.strip()
        
        # Check for trailing whitespace
        if line != line.rstrip('\n') and line.rstrip('\n') != line.rstrip():
            analysis["issues"]["trailing_whitespace"].append(i)
        
        # Track empty lines
        if not stripped:
            analysis["statistics"]["empty_lines"] += 1
            consecutive_empty += 1
            if consecutive_empty > 2:
                analysis["issues"]["excessive_empty_lines"].append({
                    "line": i,
                    "count": consecutive_empty
                })
        else:
            consecutive_empty = 0
            
            # Detect single character lines (OCR errors)
            if len(stripped) == 1 and stripped.isalpha():
                if i > 2 and len(lines[i-2].strip()) <= 2:
                    analysis["issues"]["ocr_errors"].append({
                        "line": i,
                        "text": stripped
                    })
            
            # Count headings and check formatting
            if line.startswith("#"):
                level = len(line) - len(line.lstrip("#"))
                analysis["statistics"][f"H{level}"] += 1
                if re.search(r'\s{3,}', stripped):
                    analysis["issues"]["heading_formatting"].append(i)
            
            # Validate table formatting
            if "|" in line and line.strip().startswith("|"):
                pipe_count = line.count("|")
                if pipe_count < 3:
                    analysis["issues"]["malformed_tables"].append({
                        "line": i,
                        "pipe_count": pipe_count
                    })
            
            # Detect excessive spaces (outside tables)
            if re.search(r' {4,}', stripped) and "|" not in line:
                analysis["issues"]["excessive_spaces"].append(i)
            
            # Validate image links
            img_pattern = r'!\[([^\]]*)\]\(([^)]+)\)'
            for match in re.finditer(img_pattern, line):
                alt_text, path = match.groups()
                if not alt_text.strip():
                    analysis["issues"]["empty_image_alt"].append(i)
                if not path.startswith("images/") and not path.startswith("http"):
                    analysis["issues"]["broken_image_links"].append({
                        "line": i,
                        "path": path[:50]
                    })
    
    return analysis


def clean_markdown_based_on_analysis(markdown_path, analysis, output_path=None):
    """
    Clean markdown file based on issues identified in analysis.
    
    Returns:
        Tuple of (output_file_path, changes_dict)
    """
    markdown_path = Path(markdown_path)
    
    with open(markdown_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    cleaned_lines = []
    changes = defaultdict(int)
    skip_lines = set()
    
    # Mark corrupted lines for removal
    for issue in analysis["issues"].get("ocr_errors", []):
        skip_lines.add(issue["line"] - 1)
        changes["removed_corrupted"] += 1
    
    consecutive_empty = 0
    
    for i, line in enumerate(lines):
        if i in skip_lines:
            continue
        
        original = line
        
        # Remove trailing whitespace
        if line != line.rstrip('\n'):
            line = line.rstrip('\n').rstrip() + '\n'
            if line != original:
                changes["trailing_whitespace"] += 1
        
        # Handle empty lines (limit to max 2 consecutive)
        if not line.strip():
            consecutive_empty += 1
            if consecutive_empty <= 2:
                cleaned_lines.append(line)
            else:
                changes["excessive_empty_lines"] += 1
                continue
        else:
            consecutive_empty = 0
            
            # Normalize excessive spaces (preserve tables)
            if "|" not in line and re.search(r' {4,}', line):
                line = re.sub(r' {4,}', '   ', line)
                changes["excessive_spaces"] += 1
            
            # Fix heading spacing
            if line.strip().startswith("#"):
                heading = line.lstrip("#").strip()
                heading_fixed = re.sub(r'\s+', ' ', heading)
                if heading != heading_fixed:
                    prefix = "#" * (len(line) - len(line.lstrip("#")))
                    line = prefix + " " + heading_fixed + "\n"
                    changes["heading_spacing"] += 1
        
        cleaned_lines.append(line)
    
    # Save cleaned markdown
    if output_path is None:
        output_path = markdown_path.parent / f"{markdown_path.stem}_final.md"
    else:
        output_path = Path(output_path)
    
    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(cleaned_lines)
    
    return str(output_path), changes




In [56]:
# CELL_ID: execute_cleaning

# Apply markdown quality cleaning
print("="*60)
print("STEP 3: APPLYING MARKDOWN QUALITY CLEANING")
print("="*60)

# Analyze the output from TOC-based fixing
analysis = analyze_markdown_quality(output_file)

print(f"\n📊 Quality Analysis:")
print(f"  • Total lines: {analysis['total_lines']:,}")
print(f"  • Total characters: {analysis['total_chars']:,}")
print(f"  • Empty lines: {analysis['statistics']['empty_lines']:,}")

print(f"\n📑 Heading Counts:")
for key in sorted([k for k in analysis['statistics'].keys() if k.startswith('H')]):
    print(f"  • {key}: {analysis['statistics'][key]}")

# Display issues found
print(f"\n🔍 Issues Found:")
total_issues = 0
for issue_type, issues in analysis['issues'].items():
    count = len(issues)
    if count > 0:
        print(f"  ⚠ {issue_type}: {count}")
        total_issues += count

if total_issues == 0:
    print("  ✓ No issues detected!")

# Clean the markdown
cleaned_file, cleaning_changes = clean_markdown_based_on_analysis(output_file, analysis, output_path=None)

print(f"\n{'='*60}")
print("CLEANING COMPLETE")
print(f"{'='*60}")
print(f"\n✓ Cleaning changes:")
for change_type, count in sorted(cleaning_changes.items()):
    if count > 0:
        print(f"  • {change_type}: {count}")

print(f"\n✓ Final cleaned file saved to: {cleaned_file}")
print(f"\n📝 Output file: {cleaned_file}")


STEP 3: APPLYING MARKDOWN QUALITY CLEANING

📊 Quality Analysis:
  • Total lines: 4,150
  • Total characters: 356,708
  • Empty lines: 1,331

📑 Heading Counts:
  • H1: 18
  • H2: 38
  • H3: 31

🔍 Issues Found:
  ⚠ ocr_errors: 78
  ⚠ excessive_spaces: 3

CLEANING COMPLETE

✓ Cleaning changes:
  • excessive_empty_lines: 59
  • excessive_spaces: 3
  • removed_corrupted: 78

✓ Final cleaned file saved to: output\Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited_hierarchy_fixed_final.md

📝 Output file: output\Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1_with_images_edited_hierarchy_fixed_final.md


## 5. Detailed Matching Report


In [57]:
# CELL_ID: detailed_report

# Generate detailed comparison report
def generate_detailed_report(input_file, toc_entries, stats):
    """Generate a detailed report comparing TOC vs document headings"""
    from collections import defaultdict
    
    # Read the input file to find actual headings
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    numbered_pattern = re.compile(r'^(#{1,6})\s+((?:\d+\.)+\d+\.?)\s+(.+?)\s*$')
    
    doc_headings = {}
    for line in lines:
        match = numbered_pattern.match(line.strip())
        if match:
            number = match.group(2).strip()
            heading_text = match.group(3).strip()
            if not number.endswith("."):
                number += "."
            doc_headings[number] = heading_text
    
    # Find matches and mismatches
    text_mismatches = []
    doc_only = []
    
    for number in doc_headings:
        if number in toc_entries:
            doc_text = doc_headings[number]
            toc_title = toc_entries[number]
            # Check if text differs
            norm_doc = re.sub(r'\s+', ' ', doc_text.lower().strip())
            norm_toc = re.sub(r'\s+', ' ', toc_title.lower().strip())
            if norm_doc != norm_toc:
                text_mismatches.append((number, toc_title, doc_text))
        else:
            doc_only.append((number, doc_headings[number]))
    
    # Find TOC entries not in document
    toc_only = [(num, title) for num, title in toc_entries.items() if num not in doc_headings]
    
    return text_mismatches, doc_only, toc_only, doc_headings


print(f"{'='*60}")
print("DETAILED MATCHING REPORT")
print(f"{'='*60}")

text_mismatches, doc_only, toc_only, doc_headings = generate_detailed_report(input_file, toc_entries, stats)

print(f"\n📋 TOC Entries: {len(toc_entries)}")
print(f"📋 Document Headings: {len(doc_headings)}")
print(f"\n✓ Perfect matches (same text): {stats['matched_by_number'] - stats['matched_by_text']}")
print(f"⚠ Text mismatches (fixed): {len(text_mismatches)}")

if text_mismatches:
    print(f"\n🔍 Sample text mismatches corrected:")
    for i, (num, toc_title, doc_text) in enumerate(text_mismatches[:10]):
        print(f"  {num}")
        print(f"    TOC:  '{toc_title}'")
        print(f"    Doc:  '{doc_text}'")
    if len(text_mismatches) > 10:
        print(f"    ... and {len(text_mismatches) - 10} more")

if doc_only:
    print(f"\n⚠ Document-only headings (not in TOC): {len(doc_only)}")
    for num, text in doc_only[:15]:
        display_text = f"{text[:60]}..." if len(text) > 60 else text
        print(f"    {num}: {display_text}")

if toc_only:
    print(f"\n⚠ TOC entries not found in document: {len(toc_only)}")
    for num, title in toc_only[:15]:
        display_title = f"{title[:60]}..." if len(title) > 60 else title
        print(f"    {num}: {display_title}")


DETAILED MATCHING REPORT

📋 TOC Entries: 68
📋 Document Headings: 68

✓ Perfect matches (same text): 63
⚠ Text mismatches (fixed): 5

🔍 Sample text mismatches corrected:
  3.4.
    TOC:  'Co-Morbidities in Diabetes Mellitus'
    Doc:  'Comorbidities in Diabetes Mellitus'
  5.3.
    TOC:  'Diabetes in the Older AAdults'
    Doc:  'Diabetes in the Older adults'
  5.6.
    TOC:  'Diabetes andTB'
    Doc:  'Diabetes and TB'
  5.6.1.
    TOC:  'Effects of diabetes onTB'
    Doc:  'Effects of diabetes on TB'
  5.6.2.
    TOC:  'Effects ofTB on diabetes'
    Doc:  'Effects of TB on diabetes'
